# Word2vec实战

导入gensim中的word2vec模型

In [13]:
from gensim.models import word2vec

用生成器的方式读取文件里的句子，适合读取大容量文件，而不用加载到内存

In [14]:
class MySentences(object):
    def __init__(self, fname):
        self.fname = fname
 
    def __iter__(self):
        for line in open(self.fname, 'r'):
            yield line.split()

定义一个训练函数，指定输入的文件路径和待输出的模型路径

In [15]:
def W2vTrain(flie_input, model_output):         
    sentences = MySentences(DataDir+file_input)
    w2v_model = word2vec.Word2Vec(sentences, 
                                  min_count = MIN_COUNT, 
                                  workers = CPU_NUM, 
                                  size = VEC_SIZE,
                                  window = CONTEXT_WINDOW
                                 )
    w2v_model.save(ModelDir+model_output)

上面`word2vec.Word2Vec()`便是Word2vec在gensim中的实现，一些参数解释如下：

- min_count: 对于词频 < min_count 的单词，将舍弃（其实最合适的方法是用 UNK 符号代替，即所谓的『未登录词』，这里简化起见，认为此类低频词不重要，直接抛弃）
- workers: 可以并行执行的核心数，需要安装 Cython 才能起作用（安装 Cython 的方法很简单，直接 pip install cython）
- size: 词向量的维度，即Skip-gram或CBOW模型隐藏层的节点数
- window: 目标词汇的上下文单词距目标词的最长距离，很好理解，比如 CBOW 模型是用一个词的上下文预测这个词，那这个上下文总得有个限制，如果取得太多，距离目标词太远，有些词就没啥意义了，而如果取得太少，又信息不足，所以 window 就是上下文的一个最长距离

开始训练

In [16]:
DataDir = "./"
ModelDir = "./ipynb_garbage_files/"
MIN_COUNT = 5
CPU_NUM = 2 # 需要预先安装 Cython 以支持并行
VEC_SIZE = 20
CONTEXT_WINDOW = 5 # 提取目标词上下文距离最长5个词

file_input = "bioCorpus_5000.txt"
model_output = "test_w2v_model"

W2vTrain(file_input, model_output)

加载模型

In [17]:
w2v_model = word2vec.Word2Vec.load(ModelDir+model_output)

# 查找body这个词的近义词
w2v_model.most_similar('body')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


[('influence', 0.9991704821586609),
 ('in', 0.9991064667701721),
 ('membrane', 0.9990575909614563),
 ('beta', 0.9990391731262207),
 ('isolated', 0.9990122318267822),
 ('its', 0.9989871382713318),
 ('distribution', 0.9989349842071533),
 ('two', 0.9989125728607178),
 ('proceedings', 0.9989111423492432),
 ('binding', 0.9988784790039062)]

左边一列是相似词，右边是相似度，可以看到，除了blood, cardiac, plasma这些比较make sense的词外，混入了with，to，after这些明显不make sense的词，在 NLP 里面我们叫『停用词』，也就是像常见代词、介词之类，造成这种结果的原因有二：

- 参数设置不佳，比如`vec_size`设置的太小，导致这20个维度不足以capture单词间不同的信息，所以需要继续调整超参数
- 数据集较小，因此停止词占据了太多信息量

导入NLTK包，去除停用词

In [18]:
from nltk.corpus import stopwords
StopWords = stopwords.words('english')

接下来需要在读取数据集每句话之后，对该句话进行处理，再扔进word2vec，所以需要定义一个新的训练函数，再原来基础上加上去除停用词的操作，并重新训练模型（为了方便对比，参数设置跟上面一样保持不变）

In [19]:
# 重新训练，定义新的模型训练函数
def W2vTrain_removeStopWords(file_input, model_output):         
    sentences = list(MySentences(DataDir+file_input))
    for idx,sentence in enumerate(sentences):
        sentence = [w for w in sentence if w not in StopWords]
        sentences[idx]=sentence
    w2v_model = word2vec.Word2Vec(sentences, min_count = MIN_COUNT, 
                                  workers = CPU_NUM, size = VEC_SIZE)
    w2v_model.save(ModelDir+model_output)

W2vTrain_removeStopWords(file_input, model_output)

In [20]:
w2v_model = word2vec.Word2Vec.load(ModelDir+model_output)
w2v_model.most_similar('body')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('influence', 0.9866985082626343),
 ('distribution', 0.9854052066802979),
 ('membrane', 0.9842897057533264),
 ('two', 0.9840079545974731),
 ('isolated', 0.982103705406189),
 ('effects', 0.9817605018615723),
 ('muscle', 0.9813024401664734),
 ('drugs', 0.9807368516921997),
 ('beta', 0.9807223677635193),
 ('binding', 0.9806714057922363)]

In [ ]:
观察两次的结果，发现停用词的情况有所改善，但是效果依然不明显，可能原因有二，一是一些超参数需要细致的调参，二是语料库规模太小。